In [1]:
from pathlib import Path
import pandas as pd

In [2]:
PATH = Path.cwd().parent.joinpath('data')

## Import data

In [3]:
gemeenten = pd.read_csv(PATH.joinpath('gemeenten', 'gemeenten_per_jaar.csv'), dtype={'code': str})
opsplitsingen = pd.read_csv(PATH.joinpath('gemeenten', 'opsplitsingen.csv'), dtype={'code_nieuw': str,
                                                                                    'code_oud': str})
samenvoegingen = pd.read_csv(PATH.joinpath('gemeenten', 'samenvoegingen.csv'), dtype={'code_nieuw': str,
                                                                                    'code_oud': str})

onderwijs = pd.read_csv(PATH.joinpath('onderwijs', 'speciaal_onderwijs.csv'), sep=';')

print(f'Imported {len(gemeenten)} gemeenten, {len(opsplitsingen)} opsplitsingen and {len(samenvoegingen)} samenvoegingen')

Imported 4795 gemeenten, 6 opsplitsingen and 116 samenvoegingen


In [4]:
onderwijs.rename(columns={'Regiokenmerken': 'gemeente', 
                           'Leerlingen/studenten (aantal)': 'aantal_leerlingen',
                           'Perioden': 'jaar'}, inplace=True)
onderwijs.columns = onderwijs.columns.str.lower()
onderwijs.jaar = onderwijs.jaar.str[0:4].astype(int)
onderwijs.dropna(subset='aantal_leerlingen', inplace=True)
onderwijs.aantal_leerlingen = onderwijs.aantal_leerlingen.astype(int)

In [5]:
vso = onderwijs.query('onderwijssoort == "Totaal (voortgezet) speciaal onderwijs "').copy()
vso = onderwijs.query('geslacht == "Totaal mannen en vrouwen"').copy()
len(vso)

16488

In [6]:
# Define the dictionary for replacements
to_replace = {'Gaasterl‚n-Sleat': 'Gaasterlân-Sleat',
              ' (gemeente)':  '',
              ' (O.)': '',
              ' (NH.)': '',
              ' (Z.)': '',
              ' (ZH.)': '',
              ' (L.)': '',
              "'s-Hertogenbosch": 'Den Bosch',
              's-Hertogenbosch': 'Den Bosch',
              "'s-Gravenhage": 'Den Haag',
              's-Gravenhage': 'Den Haag',
              'S˙dwest-Frysl‚n': 'Súdwest-Fryslân',
              'Skarsterl‚n': 'Skarsterlân',
              }

# Replace substrings in the 'gemeente' column of vso dataframe
for key, value in to_replace.items():
    vso['gemeente'] = vso['gemeente'].str.replace(key, value).str.strip()

# Replace substrings in the 'gemeentenaam' column of gemeenten dataframe
for key, value in to_replace.items():
    gemeenten['gemeentenaam'] = gemeenten['gemeentenaam'].str.replace(key, value).str.strip()

In [7]:
vso = pd.merge(vso[['jaar', 'gemeente', 'aantal_leerlingen']],
               gemeenten,
               left_on=['jaar', 'gemeente'],
               right_on=['jaar', 'gemeentenaam'],
               how='left'
               ).drop('gemeentenaam', axis=1)

vso = vso[vso.gemeente != 'Gemeenten; niet in te delen'].copy()

In [9]:
vso

,jaar,gemeente,aantal_leerlingen,code
0,2013,Aa en Hunze,80,1680
1,2014,Aa en Hunze,50,1680
2,2015,Aa en Hunze,60,1680
3,2016,Aa en Hunze,50,1680
4,2017,Aa en Hunze,60,1680
...,...,...,...,...
16560,2019,Zwolle,7350,0193
16561,2020,Zwolle,7470,0193
16562,2021,Zwolle,7470,0193
16563,2022,Zwolle,7510,0193
